### compareModels - A Python Jupyter Notebook to explore FEL Lamp interpolation models

In [1]:
import re
import sys  
sys.path.insert(1, '../src')
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from WhiteSpline import WhiteSpline
from NIST import NIST
from SSBUV import SSBUV, SSBUVw0 
from FELinterp import readOptronicLampData

print("FELinterp - A Python package for FEL Lamp interpolation models")

FELinterp - A Python package for FEL Lamp interpolation models


In [35]:
lampSN = 'F1711'
lampSN = 'F1738'
lampSN = 'F1739'
nsamples = 50

df = readOptronicLampData(lampSN)
# Chose an arbitrary reference model
RefModel = SSBUV
Models = (NIST, WhiteSpline)

# Choose and arbitrary wavelength grid for interpolation
# w_interp = np.linspace(250.0, 1100.0, (1100-250)+1)  # Arbitrary wl grid
wavelengths = df.wavelength.values
w_interp = np.linspace(wavelengths.min(), wavelengths.max(), 1000)  # Arbitrary wl grid

# Create the reference model
# This will be used to compute relative residuals for all other models
irr_ref = RefModel(df.wavelength.to_numpy(), df.irradiance.to_numpy(), df.uncertainty_rel.to_numpy()).model(w_interp)
print(f"{RefModel.__name__} is the reference model.")


# Each comparison model will be plotted as a separate trace
# The y-axis will show the relative residuals compared to the reference model
import plotly.colors

# Define a color palette with enough distinct colors for your models
palette = plotly.colors.qualitative.Plotly
num_models = len(Models)
colors = palette[:num_models]

fig = go.Figure()
for i, model in enumerate(Models):
    print(model.__name__)
    theModel = model(df.wavelength.to_numpy(), df.irradiance.to_numpy(), df.uncertainty_rel.to_numpy())
    irr = theModel.model(w_interp)
    irr_resid = 100 * (irr - irr_ref) / irr_ref

    # Line trace (interpolated residuals)
    fig.add_trace(go.Scatter(
        x=w_interp, y=irr_resid, mode='lines',
        name=f"{model.__name__} : {RefModel.__name__}",
        line=dict(width=1.5, color=colors[i])
    ))

    # Marker trace (data residuals)
    fig.add_trace(go.Scatter(
        x=theModel.wl_data, y=theModel.residuals(), mode='markers',
        marker=dict(size=5, color=colors[i]),
        name=f"{model.__name__} : {RefModel.__name__}",
        showlegend=False, # Show legend only for the first model
        # error_y=dict(type='data', array=theModel.unc_data_rel_k2, visible=True)
    ))

fig.update_yaxes(range=[-0.6, 0.6])
fig.update_layout(
    title=f'Relative Residuals of {lampSN} - {RefModel.__name__} vs. Other Models',
    xaxis_title='Wavelength [nm]',
    yaxis_title='Relative Residual [%]',
    legend=dict(title='Models'),
)
fig.show()

Reading /Users/ericrehm/src/FELinterp/lamps/Optronic/F-1739/F1739_22.std
Reading /Users/ericrehm/src/FELinterp/lamps/Optronic/F-1739/F1739_k2uncertainty.dat
SSBUV is the reference model.
NIST
WhiteSpline


/Users/ericrehm/src/FELinterp/notebooks/../src/SSBUV.py:89: RuntimeWarning:

divide by zero encountered in power

/Users/ericrehm/src/FELinterp/notebooks/../src/SSBUV.py:82: RuntimeWarning:

overflow encountered in power



In [39]:
# Each comparison model will be plotted as a separate trace
# The y-axis will show the relative residuals compared to the reference model
import plotly.colors

# Define a color palette with enough distinct colors for your models
palette = plotly.colors.qualitative.Plotly
num_models = len(Models)
colors = palette[:num_models]

fig = go.Figure()
for i, model in enumerate(Models):
    print(model.__name__)
    theModel = model(df.wavelength.to_numpy(), df.irradiance.to_numpy(), df.uncertainty_rel.to_numpy())
    irr = theModel.model(w_interp)
    irr_resid = 100 * (irr - irr_ref) / irr_ref

    # Line trace (interpolated residuals)
    fig.add_trace(go.Scatter(
        x=w_interp, y=irr_resid, mode='lines',
        name=f"{model.__name__} : {RefModel.__name__}",
        line=dict(width=1.5, color=colors[i])
    ))

    # Marker trace (data residuals)
    fig.add_trace(go.Scatter(
        x=theModel.wl_data, y=theModel.residuals(), mode='markers',
        marker=dict(size=5, color=colors[i]),
        name=f"{model.__name__} : {RefModel.__name__}",
        showlegend=True, # Show legend only for the first model
        error_y=dict(type='data', array=theModel.unc_data_rel_k2, visible=True)
    ))

fig.update_yaxes(range=[-3.0, 3.0])
fig.update_layout(
    title=f'Relative Residuals of {lampSN} - {RefModel.__name__} vs. Other Models',
    xaxis_title='Wavelength [nm]',
    yaxis_title='Relative Residual [%]',
    legend=dict(title='Models'),
)
fig.show()

NIST
WhiteSpline


In [ ]:
# Define a color palette with enough distinct colors for your models

allModels = (NIST, WhiteSpline, SSBUV)

palette = plotly.colors.qualitative.Plotly
num_models = len(allModels)
colors = palette[:num_models+1]

fig = go.Figure()
for i, model in enumerate(allModels):
    print(model.__name__)
    theModel = model(df.wavelength.to_numpy(), df.irradiance.to_numpy(), df.uncertainty_rel.to_numpy())
    irr = theModel.model(w_interp)

    # Line trace (interpolated residuals)
    fig.add_trace(go.Scatter(
        x=w_interp, y=irr, mode='lines',
        name=f"{model.__name__}",
        line=dict(width=1.5, color=colors[i])
    ))

# Marker trace (class-derived absolute uncdertainty)
fig.add_trace(go.Scatter(
    x=theModel.wl_data, y=theModel.irr_data, mode='markers',
    marker=dict(size=5, color=colors[i+1]),
    name=f"Class uncertainty {lampSN}",
    showlegend=True, # Show legend only for the first model
    error_y=dict(type='data', array=theModel.unc_data_abs_k1*2, visible=True)
))

# fig.update_yaxes(range=[-3.0, 3.0])
fig.update_layout(
    title=f'Interpolated irradaicne {lampSN}',
    xaxis_title='Wavelength [nm]',
    yaxis_title='Irradiance',
    legend=dict(title='Models'),
    height=600,
)

NIST
WhiteSpline
SSBUV


/Users/ericrehm/src/FELinterp/notebooks/../src/SSBUV.py:89: RuntimeWarning:

divide by zero encountered in power

/Users/ericrehm/src/FELinterp/notebooks/../src/SSBUV.py:82: RuntimeWarning:

overflow encountered in power

